### titanic dataset을 활용한 n번째 탑승객이 사망했는지 예측하기

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
